In [1]:
import torch
import sys
from swift.llm import get_model_tokenizer
dirs = ["/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm"]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)
from custom import CustomModelType, CustomTemplateType
model_type = CustomModelType.mixtral_moe_7b_instruct_gptq_int4
model, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)


2024-03-30 23:00:01,634 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-03-30 23:00:01,637 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-03-30 23:00:01,673 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 ad87f4a589251d86333765b92ab59e41 and a total number of 972 components indexed
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidde

In [2]:
import jsonlines
from tqdm.notebook import tqdm

dict_list = []
version = "1"
with jsonlines.open(f"data{version}.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        dict_list.append(item)

lens1 = []
lens2 = []

for item in tqdm(dict_list):
    lens1.append(len(tokenizer(item["query"])["input_ids"]))
    # lens2.append(len(tokenizer.tokenize(item["query"])))



  0%|          | 0/12192 [00:00<?, ?it/s]

In [3]:
# 3705

# [3675, 3674, 3671, 3665, 3662, 3662, 3640, 3638, 3636, 3632, 3616, 3604]


sorted(lens1, reverse=False)[:20]


[640,
 660,
 665,
 685,
 691,
 719,
 719,
 724,
 734,
 736,
 742,
 743,
 748,
 752,
 761,
 762,
 763,
 765,
 769,
 769]

修改较长的句子

显卡 24GB

7B模型，长度控制在3600以下 


In [4]:
del_ids = []
for i, item in enumerate(lens1):
    if item == 640:
        del_ids.append(i)

del_ids


[8493]

In [5]:
# print(len(tokenizer(dict_list[4157]["query"])["input_ids"]))
dict_list[8493]["query"]


'According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-04-13\nContent: Politician Sebastian Paul called Modi haters demented.\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2020-04-13\nTitle: FALSE: Politician Sebastian Paul called Modi haters demented. - ...\nContent:\nExplanation: <strong>Sebastian</strong> <strong>Paul</strong> has clarified that these claims are baseless and he has not made any such statement. Read the Full Article (FactCrescendo)\nInformation 2:\nPublication date: 2023-09-25\nTitle: Modi’s Party Linked With Most Hate Speech in India, Report Finds ...\nContent:\nIndian Prime Minister Narendra <strong>Modi</strong>’s ruling party and affiliated groups were behind most hate speech events against Muslims during the first half of the year

In [6]:
print(dict_list[8493]["query"])


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-04-13
Content: Politician Sebastian Paul called Modi haters demented.

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2020-04-13
Title: FALSE: Politician Sebastian Paul called Modi haters demented. - ...
Content:
Explanation: <strong>Sebastian</strong> <strong>Paul</strong> has clarified that these claims are baseless and he has not made any such statement. Read the Full Article (FactCrescendo)
Information 2:
Publication date: 2023-09-25
Title: Modi’s Party Linked With Most Hate Speech in India, Report Finds ...
Content:
Indian Prime Minister Narendra <strong>Modi</strong>’s ruling party and affiliated groups were behind most hate speech events against Muslims during the first half of the year, according to a 